# Chap 01  The Python Data Model

Python的语言风格和设计思想基于它的数据模型。数据模型规范了语言自身构建模块的API，模块包括序列、迭代器、函数、类、上下文管理器等等。通过这些API，我们可以用地道的Python语言构建自己定义的类。

Python 解释器在碰到特殊的句法时，会使用一些以双下划线开头和结尾的特殊方法（魔法函数）去激活一些基本的对象操作。比如my_obj[key] 的背后就是 __getitem__ 方法，解释器实际上调用的就是my_obj.__getitem__(key)。这些特殊方法能够让自己定义的对象支持很多基本的语言结构并与之交互，比如迭代、集合类、访问属性、运算符重载、函数和方法的调用、对象的创建和销毁、字符串表示和格式化输出、上下文管理。


## A Pythonic Card Deck

In [1]:
from collections import namedtuple

# Card是一个类，包含rank和suit两个成员，通过Card(rank, suit)创建一张卡片
Card = namedtuple('Card', ['rank', 'suit'])


class FrenchDeck():
    ranks = [str(i) for i in range(2, 10)] + list('JQKA')
    suits = ['spades', 'dimonds', 'clubs', 'hearts']

    def __init__(self):
        self._cards = [Card(rank, suit) for rank in self.ranks for suit in self.suits]

    def __len__(self):
        # 让FrenchDeck类支持len()操作
        return len(self._cards)  # _cards是一个list，自然支持len

    def __getitem__(self, position):
        # 让FrenchDeck类支持索引操作
        return self._cards[position]

FrenchDeck 这个类短小精悍。它跟任何标准 Python 集合类型一样，可以用 len() 函数
查看一叠牌有多少张，或通过索引查看某一张牌。

In [2]:
deck = FrenchDeck()
print(len(deck))  # 调用deck.__len__()
print(deck[-1])  # 调用deck.__getitem(-1)

48
Card(rank='A', suit='hearts')


Python这样做有一个明显的好处。在查看序列的长度时我们只要记住使用len()即可，而不用记忆方法名，到底是.size()还是.length()。
如果我们想实现抽牌的功能，无须自己取写函数。Python已经内置了从一个序列中随机选取元素的函数在random.choice。直接把它用在这个实例上即可。


In [3]:
from random import choice
print(choice(deck))
print(choice(deck))

Card(rank='7', suit='spades')
Card(rank='8', suit='clubs')


因为FrenchDeck类中定义了__getitem__方法，于是deck支持索引操作[]，于是它自然也支持切片操作。

In [4]:
print(deck[:3])

[Card(rank='2', suit='spades'), Card(rank='2', suit='dimonds'), Card(rank='2', suit='clubs')]


定义了__getitem__方法之后，对象还是iterable的。

In [5]:
for i, card in enumerate(deck):
    print(card)
    if (i == 5):
        break

Card(rank='2', suit='spades')
Card(rank='2', suit='dimonds')
Card(rank='2', suit='clubs')
Card(rank='2', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='3', suit='dimonds')


同时它也支持反向迭代

In [6]:
for i, card in enumerate(reversed(deck)):
    print(card)
    if i == 3:
        break

Card(rank='A', suit='hearts')
Card(rank='A', suit='clubs')
Card(rank='A', suit='dimonds')
Card(rank='A', suit='spades')


如果一个集合collection中定义了__contain__，就可以使用in运算符。但这里没有定义，于是在使用in运算符时会进行一次隐式迭代搜索。而FrenchDeck是支持迭代操作的，因此in操作符依然可以用在这个类上。

In [7]:
Card('K', 'spades') in deck

True

接下来我们想要对牌进行排序。因为FrenchDeck实现了__len__和__getitem__两个特殊方法，该类就和一个Python自有的序列类型一样了，它自然也支持sorted函数。下面我们对一副扑克牌按“数字+花色”进行排序，花色由大到小依次为Spades、Hearts、Dimonds、Clubs。

In [8]:
# 对花色排大小
suit_values = dict(spades=3, hearts=2, dimonds=1, clubs=0)
print(suit_values)

{'spades': 3, 'hearts': 2, 'dimonds': 1, 'clubs': 0}


In [9]:
def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)  # card.rank是一个str
    return rank_value * len(suit_values) + suit_values[card.suit]  # 先排大小再拍花色

r1 = spades_high(Card(rank='2', suit='clubs'))
r2 = spades_high(Card(rank='3', suit='clubs'))
r3 = spades_high(Card(rank='3', suit='spades'))
r1, r2, r3

(0, 4, 7)

In [10]:
for i, card in enumerate(sorted(deck, key=spades_high)):
    print(card)
    if i == 10:
        break

Card(rank='2', suit='clubs')
Card(rank='2', suit='dimonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='dimonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='dimonds')
Card(rank='4', suit='hearts')


更进一步，如果想洗牌功能呢？目前为止FrenchDeck是immutable的，还不能洗牌。后面会介绍如何通过添加__setitem__方法实现洗牌功能。

## How Special Methods Are Used

特殊方法的存在是为了被Python解释器调用的，我们自己并不需要去调用它。说白了，你应该写len(my_obj)让解释器去调用--len--，而不自己去调用"obj.--len--()"。很多时候，特殊方法的调用是隐式的，比如在执行for i in x这个语句时，实际是在执行iter(x)，即x.--iter--()。唯一一个需要经常被显式调用的方法是--init--，用于在自己定义的子类中调用父类的构造函数。使用诸如len、iter、str等内置函数不仅可以帮助调用特殊方法，它们的执行速度也会更快。不要自己想当然地去随意添加特殊方法。

## Self-defined Vector —— Emulating Numeric Types

In [11]:
import math

class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def __repr__(self):
        # __repr__函数最好能保证返回的结果可以构造出原对象
        print('in __repr__()')
        return ('Vector({0.x}, {0.y})'.format(self))
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))  # 调用上面重写的__abs__
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)
    
    def __mul__(self, scalar):
        return Vector(self.x*scalar, self.y*scalar)
    
    def __rmul__(self, scalar):
        print('in __rmul__')
        return Vector(self.x*scalar, self.y*scalar)

In [12]:
v = Vector(3, 4)

## String Representation

内置函数repr（report的缩写）可以调用特殊函数--repr--，用于得到一个对象的字符串表达，方便检查对象。如果某对象的类中没有实现--repr--，在控制台打印对象时会得到一个类似<Vector object at 0x10e100070>的输出。--repr-- 返回的字符串应当是明确的，没有歧义的，如果有可能应当让它匹配产生该对象的源码，就像这里应当返回形如 'Vector(a, b)' 的形式。

In [13]:
repr(v)

in __repr__()


'Vector(3, 4)'

In [14]:
eval(repr(v))

in __repr__()
in __repr__()


Vector(3, 4)

In [15]:
v

in __repr__()


Vector(3, 4)

对于更传统地采用 % 进行输出，使用 %r 则系统自动调用 repr。而对于更新的采用format进行格式化输出，使用 !r 则系统自动调用 repr。

In [16]:
print('%r' % v)

in __repr__()
Vector(3, 4)


In [17]:
print('{!r}'.format(v))

in __repr__()
Vector(3, 4)


对比--repr--和--str--，--str--是通过内值函数str()调用的。在使用print函数打印对象时会调用它。--str--更偏向于为用户展示信息，而--repr--更偏向于为程序员展示信息。如果选一个，请选择实现--repr--，因为即使未定义--str--，Python也会调用--repr--。

In [18]:
print(v.__str__())
print(str(v))
print(v)

in __repr__()
Vector(3, 4)
in __repr__()
Vector(3, 4)
in __repr__()
Vector(3, 4)


## Arithmetic Operations

Vector类通过实现了--add--和--mul--，使得对象支持 + 和 * 的操作。这两个运算符被称为中缀运算符 (infix operators)，它的基本原则就是不改变操作对象，只是通过读取它们产生一个新的值。如果想实现乘法交换律，即 3 * v，就必须在类中实现 --rmul--。

## Boolean Value of a Custom Type

Python通过内置函数bool()判断一个对象的布尔值是True还是False。在编程时通过布尔值处理上下文运算很常见的，比如if或while语句，或者and，or和not运算。默认情况下，自定义类的实例总被认为是True的，除非在类中实现了--bool--或--len--。如果--bool--方法未实现，Python会尝试调用x.--len--()，如果返回值为0，则布尔值为False，否则为True。

In [19]:
print(bool(v))
print(v.__bool__())

True
True


## Summary

    --getitem--  ==>  [], slicing, iterable
    --getitem-- + --len--  ==>  in, sorted
    --contain--  ==> in
    --repr--  ==>  repr(), {!r}.format()
    --str--  ==>  str(), 未实现时转而调用--repr--
    --bool--  ==> bool()  未实现时：默认bool(obj)为True，但若实现了--len--，当len(obj)为0是为False
    --add--  ==> +
    --radd--
    --rmul--
